In [6]:
#based on code from https://github.com/emarco177/ice_breaker

import os
from dotenv import load_dotenv

from langchain import hub
from langchain.agents import create_react_agent, AgentExecutor
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [7]:
#tools and template

def get_profile_url_and_content(name: str):
    """Search GitHub using Tavily and return the best match (url + content)."""
    search = TavilySearchResults()
    results = search.invoke(f"{name} GitHub site:github.com")

    if results and isinstance(results, list):
        best = results[0]
        return best["url"], best["content"]
    return None, None


def summarize_profile(info: str) -> str:
    """Uses LLM to summarize profile text and extract two interesting facts."""
    llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

    summary_template = """
    Given the following GitHub profile content:
    {information}

    Return:
    1. A short summary of the person
    2. Two interesting facts about them
    """

    prompt_template = PromptTemplate(
        input_variables=["information"], template=summary_template
    )

    chain = prompt_template | llm | StrOutputParser()
    return chain.invoke({"information": info})


if __name__ == "__main__":
    load_dotenv()
    print("Using LangChain with Tavily and OpenAI to find a person's GitHub URL and summarize their public profile.")
    name = input("Enter the person's name and location (e.g., 'John Mansfield Ithaca NY'): ")
    github_url, profile_content = get_profile_url_and_content(name)

    if github_url and profile_content:
        print(f"Found GitHub URL: {github_url}\n")
        summary = summarize_profile(profile_content)
        print(summary)
    else:
        print("No GitHub profile found.")

Using LangChain with Tavily and OpenAI to find a person's GitHub URL and summarize their public profile.
Enter the person's name and location (e.g., 'John Mansfield Ithaca NY'): John Mansfield, Ithaca, NY
Found GitHub URL: https://github.com/jlm429

### Summary
John Mansfield, known by his GitHub username jlm429, is a developer based in Ithaca, NY. He is active on GitHub, where he has garnered a small following and is involved in various projects.

### Interesting Facts
1. John has a LinkedIn profile under the name "john-mansfield," indicating a professional presence beyond GitHub.
2. He is affiliated with Cornell University, as suggested by his email address, which may imply a connection to academic or research activities.


In [ ]:
#TODO: add output parsers for frontend consumption